# Diplomatura en ciencia de datos, aprendizaje automático y sus aplicaciones - Edición 2023 - FAMAF (UNC)

## Análisis y visualización de datos

### Trabajo práctico entregable - Grupo 22 - Parte 2

**Integrantes:**
- Chevallier-Boutell, Ignacio José
- Ribetto, Federico Daniel
- Rosa, Santiago

**Seguimiento:** Meinardi, Vanesa

---

## Librerías

In [1]:
import io
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy
from scipy.stats import ttest_ind as Test

pd.set_option('display.max_rows', 1000) # cambiar el número de filas que se mostrarán usando display.max_rows.
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.options.mode.chained_assignment = None  # default='warn'

sns.set_context('talk')
sns.set_theme(style='white')

## Lectura del dataset

El dataset a utilizar es la encuesta Sysarmy del año 2022 versión 2, en formato csv, la cual es una una encuesta personal y voluntaria que busca relevar información sobre salarios y condiciones de trabajo de programadores, que se realiza anualmente. Se analizarán sólo los datos provenientes de Argentina. 

Se utilizará un dataset que ya ha tenido un pretratamiento: 
* Se eliminó el encabezado de la encuesta original.
* Se renombraron las columnas.

Este dataset está disponible en internet, desde donde lo usaremos.

In [2]:
url = 'https://raw.githubusercontent.com/DiploDatos/AnalisisyVisualizacion/master/sysarmy_survey_2022_processed.csv'
df = pd.read_csv(url)
total_ans = len(df) # cantidad de respuestas en tel dataset

---
# Ejercicio 1 - Estimación

## Creación del DataFrame a utilizar

Vamos a basarnos en el mismo DataFrame que utilizamos para la parte 1 del entregable. Sus características son:
- No contiene valores faltantes.
- Contiene columnas renombradas para facilitar su manipulación.
- Está filtrada considerando un sueldo bruto estrictamente mayor al sueldo neto.
- Está filtrada considerando un sueldo neto entre un Salario Mínimo Vital y Móvil (SMVM) de \$ 38.940 (Abril de 2022) y 2 millones de pesos.
- Contiene el 85 % central de la distribución.
- El género está recategorizado dentro de varón cis, mujer cis y diversidades.

In [3]:
print(f'Cantidad inicial de filas: {total_ans}.')

# Selección de columnas
relevant_columns = ["salary_monthly_BRUTO",
                    "salary_monthly_NETO",
                    "profile_gender"]

df1 = df[relevant_columns]

# Eliminación de missing values
df1 = df1.dropna(subset=relevant_columns)

# Modificación de etiquetas a usar
df1.rename(columns = {"salary_monthly_BRUTO":'bruto', 
                      "salary_monthly_NETO":'neto',
                      "profile_gender":'genero'}
                      , inplace = True)

# Bruto mayor al neto
df1 = df1[df1["bruto"]>df1["neto"]]

# Entre un SMVM y 2 millones
SMVM = 38940
df1 = df1[df1["neto"]>=SMVM]
df1 = df1[df1["neto"]<=2*1e6]

# Dejar de lado la variable bruto
relevant_columns = ["neto", "genero"]
df1 = df1[relevant_columns]

# Tomar el 85% central de la distribución
k = 15
percentile_inf = df1["neto"].quantile(k * 0.5 / 100)
percentile_sup = df1["neto"].quantile((100 - k * 0.5) / 100)

df1 = df1[df1["neto"] > percentile_inf]
df1 = df1[df1["neto"] < percentile_sup]

# Recategorizar la variable género
df1.loc[:,'genero'] = df1.genero.replace(
    {'Varón Cis': 'varon_cis',
     'Varón cis': 'varon_cis',
     'Mujer': 'mujer_cis',
     'Mujer Cis': 'mujer_cis',
     'Mujer cis': 'mujer_cis',
     'Femenino': 'mujer_cis',
     'mujer': 'mujer_cis',
     'Mujer':'mujer_cis',
     'Queer':'diversidades',
     'Varón Trans':'diversidades',
     'No binarie':'diversidades',
     'Mujer Trans':'diversidades',
     'Fluido':'diversidades',
     'Bigénero':'diversidades',
     'Gay':'diversidades'
    })

print(f'Quedan {len(df1)} filas.')

display(df1[['genero', 'neto']].describe().T)
display(df1[['genero', 'neto']].groupby('genero').describe())

Cantidad inicial de filas: 5358.
Quedan 3481 filas.


,count,mean,std,min,25%,50%,75%,max
neto,3481.0,223444.903215,87659.716775,91600.0,155000.0,214270.0,270031.0,499886.0


neto                                                                              
               count           mean           std      min       25%       50%       75%       max
genero                                                                                            
diversidades    53.0  204218.270755  73934.335271  92000.0  144000.0  190000.0  253892.0  450000.0
mujer_cis      583.0  203723.703310  81025.162448  91983.0  140000.0  195000.0  245000.0  497000.0
varon_cis     2845.0  227844.365803  88618.445676  91600.0  158944.0  220000.0  280000.0  499886.0

## Definiciones para la estadística

Vamos a separar nuestra población original en dos poblaciones: la población P1 de varones cis y la población P2 de otros géneros. Asumimos que ambas poblaciones distribuyen normalmente.
$$
P1 \sim \mathcal{N} \left( \mu_1, \sigma^2_1 \right) \quad ; \quad P2 \sim \mathcal{N} \left( \mu_2, \sigma^2_2 \right)
$$


De dicha población original tenemos una muestra de $n=3481$ elementos: $n_1=2845$ constituyen la muestra M1 de la población P1, mientras que los otros $n_2=636$ constituyen la muestra M2 de la población P2. Notamos que ambas muestras son grandes ($n_1, n_2 > 30$) e independientes. A cada muestra le calculamos su media, su desviación estándar y su varianza:
$$
\overline{m}_1 = \$\ 227.844 \quad ; \quad s_1 = \$\ 88.618 \quad ; \quad s_1^2 = \$^2\ 7.853.228.914 \\
\overline{m}_2 = \$\ 203.765 \quad ; \quad s_2 = \$\ 80.404 \quad ; \quad s_2^2 = \$^2\ 6.464.777.223 \\
$$

Nuestro objetivo es utilizar la información de las muestras para:
- Hacer una estimación puntual sobre la diferencia entre las medias poblacionales.
- Hacer una estimación por intervalos sobre la diferencia entre las medias poblacionales para determinar un intervalo de confianza de nivel (1-$\alpha$).

In [4]:
# Separación de la muestra según sea o no varón cis.
man = df1.genero == 'varon_cis'
M1 = df1[man].neto
n1 = len(M1)
m1 = M1.mean()
s1 = M1.std()

print('Descripción estadística de la muestra M1:')
print(f'\t > Tiene {n1} elementos.')
print(f'\t > Tiene una media igual a $ {m1:.0f}.')
print(f'\t > Tiene una desviación estándar igual a $ {s1:.0f}.')
print(f'\t > Tiene una varianza igual a {s1**2:.0f} pesos cuadrados.')
print('-------------------------------------------------------------------------')

M2 = df1[~man].neto
n2 = len(M2)
m2 = M2.mean()
s2 = M2.std()

print('Descripción estadística de la muestra M2:')
print(f'\t > Tiene {n2} elementos.')
print(f'\t > Tiene una media igual a $ {m2:.0f}.')
print(f'\t > Tiene una desviación estándar igual a $ {s2:.0f}.')
print(f'\t > Tiene una varianza igual a {s2**2:.0f} pesos cuadrados.')

Descripción estadística de la muestra M1:
	 > Tiene 2845 elementos.
	 > Tiene una media igual a $ 227844.
	 > Tiene una desviación estándar igual a $ 88618.
	 > Tiene una varianza igual a 7853228914 pesos cuadrados.
-------------------------------------------------------------------------
Descripción estadística de la muestra M2:
	 > Tiene 636 elementos.
	 > Tiene una media igual a $ 203765.
	 > Tiene una desviación estándar igual a $ 80404.
	 > Tiene una varianza igual a 6464777223 pesos cuadrados.


## Estimación puntual

Como la media muestral $\overline{m}_1$ de la muestra M1 es un buen estimador de la media poblacional $\mu_1$ de la población P1 y la media muestral $\overline{m}_2$ de la muestra M2 es un buen estimador de la media poblacional $\mu_2$ de la población P2, resulta razonable que para estimar la diferencia $\mu  = \mu_1 - \mu_2$ recurramos al estimador $\overline{m} = \overline{m}_1 - \overline{m}_2$. Se sigue entonces que la estimación puntual buscada es
$$
\mu \sim \overline{m} = \$\ 24.079
$$

Esto quiere decir que el salario neto promedio de los varones cis es $ 24.079 mayor respecto al salario neto promedio de los otros géneros.

In [5]:
# Diferencia de medias
m = m1 - m2
m

24079.44853901252

## Estimación por intervalos

Dado que ambas muestras son grandes, las varianzas de ambas poblaciones son desconocidas y no tenemos razones para asumir que son iguales, el pivote estadístico a utilizar será
$$
S = \sqrt{\dfrac{s_1^2}{n_1} + \dfrac{s_2^2}{n_2}} \Rightarrow T = \dfrac{\overline{m} - \mu}{S} \stackrel{Asint}{\sim} \mathcal{N} \left( 0, 1 \right)
$$

A partir de esto, se puede ver que el invervalo de confianza para $\mu$ a un nivel de confianza (1-$\alpha$) queda determinado por
$$
\overline{m} \pm z_{\alpha/2} S
$$

donde $z_{\alpha/2}$ es el $\mathcal{Z}$-score asociado, el cual es bilateral. Tomando un nivel de significancia $\alpha=0.05$, tenemos $z_{0.025} = 1.96$.

Vemos entonces que el intervalo de confianza buscado es
$$
\left[\ \$\ 24.055\  ;\ \$\ 24.104\  \right]
$$

Considerando que la incertidumbre debería tener una única cifra significativa, tendríamos $\$\ 25 \Rightarrow \$\ 20$, con lo cual las estimaciones nos quedarían:
$$
\left[\ \$\ 24.060\  ;\ \$\ 24.100\  \right] \Rightarrow \$\ \left( 24.080 \pm 20 \right)
$$

En otros términos, podemos decir que estamos un 95% seguros de que la diferencia entre las medias poblacionales cae en el intervalo $\left[\ \$\ 24.060\  ;\ \$\ 24.100\  \right]$, *i.e.* estamos un 95% seguros de que el salario neto promedio de los varones cis es entre \$ 24.060 y \$ 24.100 mayor que el salario neto promedio de otros géneros.

In [6]:
S = np.sqrt((s1/n1) + (s2/n2))
z_0025 = 1.96

pm = z_0025 * S
print(f'Incertidumbre: $ {pm:.0f}')
LI = m - pm
LS = m + pm

print('Intervalo de confianza:')
print(f'\t > Límite inferior: $ {LI:.0f}')
print(f'\t > Límite superior: $ {LS:.0f}')

Incertidumbre: $ 25
Intervalo de confianza:
	 > Límite inferior: $ 24055
	 > Límite superior: $ 24104


## Relación entre el IC y test de hipótesis

Considerando un test de hipótesis, el intervalo de confianza dado por $(-z_{\alpha/2}\ ;\ z_{\alpha/2})$ representa la región de **no** rechazo de la hipótesis nula.

---
# Ejercicio 2 - Test de hipótesis

## Formalización

Planteamos las siguientes hipótesis:
- **Hipótesis nula:** el salario neto medio de varones cis es igual al salaro neto medio de otros géneros.
$$
H_0: \mu = 0
$$
- **Hipótesis alternativa:** los varones cis no tienen un salario neto promedio igual a aquel de los otros géneros, considerando un nivel de significancia del 5%.
$$
H_1: \mu \neq 0 \ @\  \alpha=0.05
$$

Como la prueba es bilateral, la región crítica es $(-\infty\ ;\ - 1.96] \cup [1.96\ ;\ \infty)$. El estadístico de prueba a utilizar para llevar a cabo el test de hipótesis es idéntico al pivote estadístico antes planteado, tomando el $\mu$ considerado en nuestra $H_0$.
$$
S = \sqrt{\dfrac{s_1^2}{n_1} + \dfrac{s_2^2}{n_2}} \Rightarrow T = \dfrac{\overline{m}}{S} \stackrel{Asint}{\sim} \mathcal{N} \left( 0, 1 \right)
$$

Desde una perspectiva de valores críticos, como $T=1918$, el estadístico cae en la región de rechazo, lo cual nos lleva a rechazar $H_0$: con un nivel de significancia del 5%, los datos proporcionan evidencia suficiente para concluir que los salarios netos promedios entre varones cis y otros géneros **no** son iguales.

In [7]:
T = m/S
T

1918.270345654361

## Enfoque desde el $p$-valor

En este caso donde tenemos una prueba bilateral, el $p$-valor nos indica la probabilidad de que nuestro estadístico $T$ sea menor a -1918 o mayor a 1918 bajo $H_0$.

In [8]:
Test(M1, M2, equal_var=False, alternative='two-sided')

Ttest_indResult(statistic=6.697758506240167, pvalue=3.5080024073278474e-11)

Ya tenemos nuestras hipótesis planteadas y sabemos que nuestro estadístico vale $T=1918$. El área truncada sobre la cola derecha de la distribución de T dado este estadístico es prácticamente nula. En este caso no es relevante, pero al tratarse de una prueba bilaetral deberíamos duplicar este valor de área. Tenemos entonces que el $p$-valor es prácticamente igual a 0. Como $p\text{-valor} \approx 0 < 0.05 = \alpha$, rechazamos $H_0$. Nuevamete, los datos proveen evidencia suficiente de que, con un nivel de significancia del 5%, los salarios netos promedios entre varones cis y otros géneros **no** son iguales.